#Top

[![Open in GitHub](https://img.shields.io/badge/Open%20Folder%20in-GitHub-181717?logo=github&logoColor=white)](https://github.com/lindsayalexandra14/ds_portfolio/tree/main/1_projects/statistical_analysis/a_b_testing/fishers_exact_test)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LWFEoJVm84Q7L33JBShM4n1jbMwN9_yX#scrollTo=-mlchltjHZ84)

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/AB%20Testing%20Small%20Sample%20Size.png)

**summary**
*   This hypothetical experiment tests two Landing Pages (control vs. treatment)
*   The initial sample size is 30,000 users but the test gets cut short and our sample is cut to 305 users and I need to use Fisher's Exact Test for small sample sizes where a cell has < 10 users
*   The landing page needs to be switched over sooner due to changed timelines, so the results give as much information as I will get about how the treatment performed
*   I am trying to prove that the treatment performed better than the control because the team is interested in moving forward with the treatment
*  It was established from the test that the treatment performed better with significance (at alpha=0.05). The practical significance is low (cohen's h = 0.02). It did not have the full desired statistical power (75% vs. 80%)
*  In this case, given the constraints, I am comfortable enough with the treatment performing some level higher than the control with significance and not vice versa that I will recommend moving forward with implementing the treatment

**tl;dr for results**

*   Skip to "Results Summary" at the end





![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Fishers%20R.png)

#Setup

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Setup.png)

##Install packages

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Install%20Packages.png)

In [ ]:
install.packages("exact2x2")
install.packages("statmod")

##Import Libraries

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Import%20Libraries.png)

In [ ]:
library(exact2x2)
library(statmod)
library(glue)

#Test Design

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Test%20Design.png)

##Parameters

In [ ]:
alpha <- 0.05             # Significance level
power <- 0.80             # Statistical power (Probability of detecting an effect when it exists; 0.8 is standard)
control=0.14              # Baseline rate
effect <- 0.05            # Desired relative effect (e.g., 5% lift over baseline)
mde <- control * effect   # Minimum Detectable Effect (MDE) # Minimum difference to detect in absolute terms
treatment= control + mde  # Treatment rate (includes effect)
print(paste('Control:',control))
print(paste('Treatment:',treatment))

[1] "Control: 0.14"
[1] "Treatment: 0.147"


In [ ]:
p_1=treatment
p_2=control
p1_label = "Treatment"
p2_label = "Control"

alternative = "greater" # in reference to p1:
# p1 is "greater" than p2
# p1 is "less" than p2
# p1 is different from ()"two.sided" p2

hypothesis <- switch(alternative,
  greater = sprintf("%s (%.4f) is greater than %s (%.4f)", p1_label, p_1, p2_label, p_2),
  less = sprintf("%s (%.4f) is less than %s (%.4f)", p1_label, p_1, p2_label, p_2),
  two.sided = sprintf("%s (%.4f) is different from %s (%.4f)", p1_label, p_1, p2_label, p_2),
)

cat("Hypothesis:",hypothesis)


Hypothesis: Treatment (0.1470) is greater than Control (0.1400)

##Effect size

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Effect%20Size.png)

In [ ]:
# Cohen's h (standardized effect size for proportions)
proportion_effectsize <- function(treatment, control) {
  2 * asin(sqrt(treatment)) - 2 * asin(sqrt(control))
}
effect_size <- proportion_effectsize(treatment, control)

cat(sprintf("Control = %.4f\n", control))
cat(sprintf("Treatment= %.4f\n", treatment))
cat(sprintf("Minimum Detectable Effect (MDE): %.3f\n", mde))
cat(sprintf("Effect Size (Cohen's h): %.3f\n", effect_size))

Control = 0.1400
Treatment= 0.1470
Minimum Detectable Effect (MDE): 0.007
Effect Size (Cohen's h): 0.020


Cohen's h benchmarks:

0.2 = small effect

0.5 = medium effect

0.8 = large effect

If the effect is tiny, it will require a very large sample size to detect.

*   Effect is translated into Cohen’s h
*   It is a way to quantify how big the difference between two proportions is, on a standardized scale  
*   Absolute differences (like +2%) are different on a baseline of 5% vs 50%
*   Puts differences on a common scale, to compare effect sizes fairly across experiments
*   Demonstrates practical meaning (vs. just statistical significance)

##Sample Size

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Sample%20Size.png)

Calculate minimum sample size for each group (cell) for one-sided and two-sided tests:
*   A one-sided test is used when you want to test if one group performs specifically better or worse than the other (a directional hypothesis).
*   A two-sided test is used when you want to test if there is any difference between the groups, regardless of direction — whether one is better or worse.

In [ ]:
simulate_fisher_power <- function(p1, p2, n1, n2, alpha, reps = 1000, alternative, seed=100) {
  set.seed(seed)
  rejects <- replicate(reps, {
    x1 <- rbinom(1, n1, p1)
    x2 <- rbinom(1, n2, p2)

    # Creates contingency table, x1 is reference for hypothesis
    tbl <- matrix(c(x1, n1 - x1, x2, n2 - x2), nrow = 2, byrow = TRUE)

    fisher.test(tbl, alternative = alternative)$p.value < alpha
  })

  mean(rejects)
}

Option 1: Test out different #s manually to find optimal sample size to reach power:

In [ ]:
n_1 <- 30000   # input sample size for group 1
n_2 <- 30000   # input sample size for group 2

# Estimate Power at a Fixed Sample Size:
estimated_power <- simulate_fisher_power(p1=p_1, p2=p_2, n_1, n_2, alpha=alpha, alternative = alternative)
cat(sprintf("Power for manual estimate: %.3f\n", estimated_power))

Power for manual estimate: 0.800


Option 2: Use function to find the sample size with the selected power:

In [ ]:
# Minimum Sample Size Finder
find_min_sample_size <- function(p1, p2, alpha, power, max_n = 50000,
                                 reps = 1000, alternative, step = 1000, seed=100) {
  set.seed(seed)
  for (n in seq(1000, max_n, by = step)) {
    sim_power <- simulate_fisher_power(p1, p2, n1 = n, n2 = n,
                                       alpha = alpha, reps = reps, alternative = alternative, seed=seed)
    cat(sprintf("n = %d → power = %.3f\n", n, sim_power))
    if (sim_power >= power) {
      return(n)
    }
  }
  return(NA)
}

set.seed(100)
# Run the minimum sample size search
cat("\nSearching for minimum required sample size...\n")
min_n <- find_min_sample_size(p1 = p_1, p2 = p_2, alpha = alpha, power = power, alternative = alternative)
cat(sprintf("\nMinimum sample size per group to achieve %.0f%% power: %d\n", power * 100, min_n))



Searching for minimum required sample size...
n = 1000 → power = 0.108
n = 2000 → power = 0.165
n = 3000 → power = 0.204
n = 4000 → power = 0.196
n = 5000 → power = 0.261
n = 6000 → power = 0.285
n = 7000 → power = 0.334
n = 8000 → power = 0.346
n = 9000 → power = 0.370
n = 10000 → power = 0.416
n = 11000 → power = 0.448
n = 12000 → power = 0.444
n = 13000 → power = 0.492
n = 14000 → power = 0.506
n = 15000 → power = 0.534
n = 16000 → power = 0.574
n = 17000 → power = 0.584
n = 18000 → power = 0.606
n = 19000 → power = 0.596
n = 20000 → power = 0.616
n = 21000 → power = 0.648
n = 22000 → power = 0.653
n = 23000 → power = 0.700
n = 24000 → power = 0.716
n = 25000 → power = 0.723
n = 26000 → power = 0.731
n = 27000 → power = 0.747
n = 28000 → power = 0.780
n = 29000 → power = 0.776
n = 30000 → power = 0.800

Minimum sample size per group to achieve 80% power: 30000


#Results

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Results.png)

##Data

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Data.png)

###Import Data

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Import%20Data.png)

From a dataset:

In [ ]:
#df_data=

Manually input:

In [ ]:
control_conversions=7
treatment_conversions=18
control_no_conversions=150
treatment_no_conversions=130

In [ ]:
print(p1_label) # set above in test design
print(p2_label)

[1] "Treatment"
[1] "Control"


##Contingency Table

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Contingency%20Table.png)

In [ ]:
table <- matrix(c(control_conversions, control_no_conversions, treatment_conversions, treatment_no_conversions), nrow = 2, byrow = TRUE)
colnames(table) <- c("Converted", "Not_Converted")
rownames(table) <- c("Control", "Treatment")
print(table)

          Converted Not_Converted
Control           7           150
Treatment        18           130


In [ ]:
# Flip rows if p1_label is not in the first row
if (rownames(table)[1] != p1_label) {
  table_to_use <- table[c(2, 1), ]  # flip row order
}

# Print the final table
print(table_to_use)

          Converted Not_Converted
Treatment        18           130
Control           7           150


##Conversion Rates:

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Conversion%20Rates.png)

In [ ]:
n1 <- sum(table_to_use[1, ])           # Reference group (row 1)
n2 <- sum(table_to_use[2, ])           # (row 2)

p1 <- table_to_use[1, "Converted"] / n1   # Reference group (row 1) conversion rate
p2 <- table_to_use[2, "Converted"] / n2   # (row 2) conversion rate

groups <- c(p1 = p1_label, p2 = p2_label)

print(glue("p1: ","{groups['p1']} Conversion Rate: {round(p1 * 100, 2)}%"))
print(glue("p2: ","{groups['p2']} Conversion Rate: {round(p2 * 100, 2)}%"))


p1: Treatment Conversion Rate: 12.16%
p2: Control Conversion Rate: 4.46%


In [ ]:
result_hypothesis <- switch(alternative,
  greater = sprintf("%s (%.4f) is greater than %s (%.4f)", p1_label, p1, p2_label, p2),
  less = sprintf("%s (%.4f) is less than %s (%.4f)", p1_label, p1, p2_label, p2),
  two.sided = sprintf("%s (%.4f) is different from %s (%.4f)", p1_label, p1, p2_label, p2),
)

cat("Result Hypothesis:",result_hypothesis)

Result Hypothesis: Treatment (0.1216) is greater than Control (0.0446)

##Effect Size:

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Effect%20Size.png)

In [ ]:
# Absolute Difference
abs_diff <- abs(p1 - p2)

# Cohen's h function
proportion_effectsize <- function(control, treatment) {
  2 * asin(sqrt(treatment)) - 2 * asin(sqrt(control))
}

h <- proportion_effectsize(control, treatment)

cat(sprintf("Absolute difference: %.3f (%.1f%%)\n", abs_diff, abs_diff * 100))
cat(sprintf("Cohen's h: %.3f\n", h))

# Interpret effect size
interpret_h <- function(h) {
  if (abs(h) < 0.2) return("negligible")
  if (abs(h) < 0.5) return("small")
  if (abs(h) < 0.8) return("medium")
  return("large")
}
cat(sprintf("Effect size interpretation: %s\n", interpret_h(h)))


Absolute difference: 0.077 (7.7%)
Cohen's h: 0.020
Effect size interpretation: negligible


##Fisher's Exact Test

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Fishers%20Exact%20Test.png)

Use Fisher's Exact Test since a cell (control converted) has < 10 users:

In [ ]:
print(table_to_use)

          Converted Not_Converted
Treatment        18           130
Control           7           150


Check parameters and change if needed:

In [ ]:
print(alternative)
print(alpha)
print(power)
# alternative="greater" # Row 1 (reference) of Contingency Table is greater than Row 2
# alpha=0.05
# power=0.8

[1] "greater"
[1] 0.05
[1] 0.8


Run test:

In [ ]:
library(exact2x2)
result <-  exact2x2(table_to_use, alternative = alternative, conf.level = 1 - alpha, tsmethod="central")
print(result)

# ALTERNATIVE:
# If you don't need CI, can do this test:
# One side of CI not valid in one-sided tests for fisher's
# result <- fisher.test(table, alternative = alternative, conf.level = 1-alpha)
# print(result)


	One-sided Fisher's Exact Test

data:  table_to_use
p-value = 0.01186
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 1.294771      Inf
sample estimates:
odds ratio 
  2.956901 



#P-value

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Pvalue.png)

In [ ]:
# Extract p-value from result
p_value <- result$p.value

print(paste("p-value: ",round(p_value,3)))

pvalue_message <- if (p_value < alpha) {
  sprintf("Because the p-value (%.3f) is less than alpha (%.3f), this result is statistically significant at the %.0f%% confidence level.",
          p_value, alpha, (1 - alpha) * 100)
} else {
  sprintf("Because the p-value (%.3f) is greater than or equal to alpha (%.3f), this result is not statistically significant at the %.0f%% confidence level.",
          p_value, alpha, (1 - alpha) * 100)
}

cat(strwrap(pvalue_message, width = 80), sep = "\n")


[1] "p-value:  0.012"
Because the p-value (0.012) is less than alpha (0.050), this result is
statistically significant at the 95% confidence level.


###If one-sided test:

##Confidence Interval

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Confidence%20Interval.png)

###"Less" Hypothesis:

The upper bound of the CI is what you want to look at for "less" — it should be < 1 for significance.

For a "less" alternative (testing if Group B < Group A):


If the upper bound < 1:
You are confident the odds in Group B are lower than in Group A.
This supports your "less" hypothesis → Result significant.

If the upper bound ≥ 1:
It’s possible Group B’s odds are not lower than Group A’s (could be equal or higher).
This does not support your "less" hypothesis → Result not significant.

###"Greater" Hypothesis:

The lower bound of the CI is what you want to look at for "greater" — it should be > 1 for significance.

For a "greater" alternative (testing if Group B > Group A):

If lower bound > 1:
With your chosen confidence level, you can say the odds of conversion in Group B are at least this much higher than in Group A.
This supports your "greater" hypothesis → Result is significant.

If lower bound ≤ 1:
You cannot confidently say that Group B’s odds are higher than Group A’s.
This does not support your "greater" hypothesis → Result is not significant.

###Interpretation:

In [ ]:
lower_ci <- result$conf.int[1]
upper_ci <- result$conf.int[2]

# Make sure p1_label and p2_label are defined
# Example:
# p1_label <- "Group A"
# p2_label <- "Group B"

if (alternative == "less") {
  if (upper_ci < 1) {
    percent_diff <- (1 - upper_ci) * 100
    sentence <- paste0(
      sprintf("95%% CI Upper Bound for Odds Ratio: %.3f.", upper_ci), "\n\n",
      sprintf("With %.0f%% confidence, %s (p₁) has lower odds of conversion than %s (p₂).", (1 - alpha) * 100, p1_label, p2_label), "\n",
      sprintf("The odds of conversion in %s are up to %.1f%% lower than in %s.", p1_label, percent_diff, p2_label), "\n",
      sprintf("This supports the hypothesis that %s is better than %s.", p2_label, p1_label)
    )
  } else {
    sentence <- paste0(
      sprintf("95%% CI Upper Bound for Odds Ratio: %.3f.", upper_ci), "\n\n",
      sprintf("With %.0f%% confidence, we cannot rule out that %s is not better than %s.", (1 - alpha) * 100, p2_label, p1_label), "\n",
      sprintf("This does not support the hypothesis that %s is better.", p2_label)
    )
  }

} else if (alternative == "greater") {
  if (lower_ci > 1) {
    percent_diff <- (lower_ci - 1) * 100
    sentence <- paste0(
      sprintf("95%% CI Lower Bound for Odds Ratio: %.3f.", lower_ci), "\n\n",
      sprintf("With %.0f%% confidence, %s (p₁) has higher odds of conversion than %s (p₂).", (1 - alpha) * 100, p1_label, p2_label), "\n",
      sprintf("The odds of conversion in %s are at least %.1f%% higher than in %s.", p1_label, percent_diff, p2_label), "\n",
      sprintf("This supports the hypothesis that %s is better than %s.", p1_label, p2_label)
    )
  } else {
    sentence <- paste0(
      sprintf("95%% CI Lower Bound for Odds Ratio: %.3f.", lower_ci), "\n\n",
      sprintf("With %.0f%% confidence, we cannot rule out that %s is not better than %s.", (1 - alpha) * 100, p1_label, p2_label), "\n",
      sprintf("This does not support the hypothesis that %s is better.", p1_label)
    )
  }

} else if (alternative == "two.sided") {
  if (lower_ci > 1) {
    percent_diff <- (lower_ci - 1) * 100
    sentence <- paste0(
      sprintf("95%% CI: [%.3f, %.3f].", lower_ci, upper_ci), "\n\n",
      sprintf("With %.0f%% confidence, there is a statistically significant difference in odds of conversion between %s (p₁) and %s (p₂).", (1 - alpha) * 100, p1_label, p2_label), "\n",
      sprintf("The confidence interval suggests that the odds of conversion in %s may be at least %.1f%% higher than in %s.", p1_label, percent_diff, p2_label)
    )
  } else if (upper_ci < 1) {
    percent_diff <- (1 - upper_ci) * 100
    sentence <- paste0(
      sprintf("95%% CI: [%.3f, %.3f].", lower_ci, upper_ci), "\n\n",
      sprintf("With %.0f%% confidence, there is a statistically significant difference in odds of conversion between %s (p₁) and %s (p₂).", (1 - alpha) * 100, p1_label, p2_label), "\n",
      sprintf("The confidence interval suggests that the odds of conversion in %s may be up to %.1f%% lower than in %s.", p1_label, percent_diff, p2_label)
    )
  } else {
    sentence <- paste0(
      sprintf("95%% CI: [%.3f, %.3f].", lower_ci, upper_ci), "\n\n",
      sprintf("With %.0f%% confidence, we cannot rule out no difference in odds between %s and %s.", (1 - alpha) * 100, p1_label, p2_label), "\n",
      "This does not support a statistically significant difference."
    )
  }
}

cat(sentence, "\n\n")

# Significance check
includes_one <- (lower_ci <= 1) && (upper_ci >= 1)

message <- if (includes_one) {
  sprintf("Because the interval includes 1, this result is not statistically significant at the %.0f%% confidence level.", (1 - alpha) * 100)
} else {
  sprintf("Because the interval does not include 1, this result is statistically significant at the %.0f%% confidence level.", (1 - alpha) * 100)
}

cat(message)


95% CI Lower Bound for Odds Ratio: 1.295.

With 95% confidence, Treatment (p₁) has higher odds of conversion than Control (p₂).
The odds of conversion in Treatment are at least 29.5% higher than in Control.
This supports the hypothesis that Treatment is better than Control. 

Because the interval does not include 1, this result is statistically significant at the 95% confidence level.

##Statistical Power

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Statistical%20Power.png)

In [ ]:
print(table_to_use)

          Converted Not_Converted
Treatment        18           130
Control           7           150


In [ ]:
# Calculate power (one-sided test)
set.seed(100)
result_power <- power.fisher.test(n1 = n1, n2 = n2, p1 = p1, p2 = p2,
                           alpha = alpha,
                           alternative = alternative,   # Group 1 vs. Group 2
                           nsim = 10000)  # Number of simulations

In [ ]:
power_pct <- sprintf("%.1f", result_power * 100)
cat("Result Power:",power_pct,"%\n\n")

if (result_power < 0.8) {
  power_sentence <- sprintf(
    "Our test was underpowered (e.g., only ~%s%% power), meaning there was \na higher chance we failed to detect a true difference due to limited sample size. \nAs a result, we cannot be statistically \nconfident in it without further data and cannot give a confident \nestimate in incremental revenue from the test.",
    power_pct
  )
} else {
  power_sentence <- sprintf(
    "Our test was adequately powered (e.g., ~%s%% power), meaning we had a \nstrong chance of detecting a true difference if one existed.",
    power_pct
  )
}

cat(power_sentence, "\n")


Result Power: 75.3 %

Our test was underpowered (e.g., only ~75.3% power), meaning there was 
a higher chance we failed to detect a true difference due to limited sample size. 
As a result, while the effect appears meaningful, we cannot be statistically 
confident in it without further data and cannot give a confident 
estimate in incremental revenue from the test. 


# Results Summary

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Results%20Summary.png)

In [ ]:
cat("\n")
print(table_to_use)
cat("\n")
print(glue("p1: ","{groups['p1']} Conversion Rate: {round(p1 * 100, 2)}%"))
print(glue("p2: ","{groups['p2']} Conversion Rate: {round(p2 * 100, 2)}%"))
cat("\n")
cat("Result Hypothesis:",result_hypothesis)
cat("\n")
cat("\n")
cat(sprintf("Absolute difference: %.3f (%.1f%%)\n", abs_diff, abs_diff * 100))
cat(sprintf("Cohen's h: %.3f\n", h))
cat(sprintf("Effect size interpretation: %s\n", interpret_h(h)))
cat("\n")
print(result)
cat(strwrap(pvalue_message, width = 80), sep = "\n")
cat("\n")
cat(sentence, "\n")
cat("\n")
cat(message, "\n")
cat("\n")
cat("Result Power:",power_pct,"%\n\n")
cat(power_sentence, "\n")


          Converted Not_Converted
Treatment        18           130
Control           7           150

p1: Treatment Conversion Rate: 12.16%
p2: Control Conversion Rate: 4.46%

Result Hypothesis: Treatment (0.1216) is greater than Control (0.0446)

Absolute difference: 0.077 (7.7%)
Cohen's h: 0.020
Effect size interpretation: negligible


	One-sided Fisher's Exact Test

data:  table_to_use
p-value = 0.01186
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 1.294771      Inf
sample estimates:
odds ratio 
  2.956901 

Because the p-value (0.012) is less than alpha (0.050), this result is
statistically significant at the 95% confidence level.

95% CI Lower Bound for Odds Ratio: 1.295.

With 95% confidence, Treatment (p₁) has higher odds of conversion than Control (p₂).
The odds of conversion in Treatment are at least 29.5% higher than in Control.
This supports the hypothesis that Treatment is better than Control. 

Because the interval does not in

#Odds Ratio Conversion

![Alt text](https://github.com/lindsayalexandra14/ds_portfolio/raw/main/2_images/templates/notebook/headers/lilac/Odds%20Ratio%20Conversion.png)

In [ ]:
print(paste("Control:",control))
print(paste("Treatment:",treatment))

control_odds=control/(1-control)
treatment_odds=treatment/(1-treatment)
odds_ratio=treatment_odds/control_odds

print(paste("Odds Ratio:",round(odds_ratio,2)))
print(paste("Lower CI:",round(lower_ci,2)))
# print(paste("Upper CI:",round(upper_ci,2)))

[1] "Control: 0.14"
[1] "Treatment: 0.147"
[1] "Odds Ratio: 1.06"
[1] "Lower CI: 1.29"


Your hypothesized Odds Ratio (OR) of 1.06 (based on Relative Rate (RR) = 0.147/0.14 =1.05)

*  Control odds = 0.14 / (1 - 0.14) = 0.1628
*  Treatment odds = 0.147 / (1 - 0.147) = 0.1724
*  Treatment odds / Control odds = 0.1724 / 0.1628 = 1.059

5% increase in probability leads to a 6% increase in odds

If your minimum meaningful effect was an odds ratio of 1.06, then:

You’d want your lower CI bound ≥ 1.06

Your result of 1.295 >1.06 → it's statistically significant, and stronger than hypothesized

If you only wanted any significant increase, then >1 is enough.

1.295 > 1.06 (hypothesized) Strong enough to confirm a 6% OR (5% RR)increase, but without power